# Introduction

In this Juptyer Notebook file, we will introduce the subpackage of _compact quantum group channels_, stored in `qittoolbox.cqgchannels` . 

In [1]:
import numpy as np
import scipy

# Import the channel methods:
from qittoolbox.cqgchannels import BC16channel, SNpluschannel

## The Jones-Wenzl projection
The Jones-Wenzl projections $p_k$ (for $O_N^+$) and $\hat{p}_k$ (for $S_N^+$) are encoded in the methods `BC16channel.get_Jones_Wenzl_projection` and `SNpluschannel.get_Jones_Wenzl_projection_NC` , where `_NC` refers to the fact that the representation theory of $S_N^+$ is connected to the category of _non-crossing_ partitions.

In [2]:
N = 5
k = 2
pk = BC16channel.get_Jones_Wenzl_projection(k,N=N)
hat_pk = SNpluschannel.get_Jones_Wenzl_projection_NC(k,N=N)

# The returned values are scipy.sparse matrices, so in order to use numpy methods, you must first turn them to dense arrays or matrices!
pk_dense = pk.todense()
hat_pk_dense = hat_pk.todense()

# Check whether these operators are truly projections
print( np.allclose(pk_dense@pk_dense,pk_dense) , np.allclose(np.conj(pk_dense.T),pk_dense) , np.allclose(hat_pk_dense@hat_pk_dense,hat_pk_dense), np.allclose(np.conj(hat_pk_dense.T), hat_pk_dense )  )

True True True True


## Kraus operators that describe the channels $\Phi_k^{(l),m}$ and $\hat{\Phi}_k^{(l),m}$

The Kraus operators $\{ E_i \}$ and $\{F_i\}$ that describe the $O_N^+$-channels $\Phi_k^{(l),m}$ and $S_N^+$-channels $\hat{\Phi}_k^{(l),m}$ can be accessed as follows:

In [3]:
k,l,m = 2,4,2
N = 5
pk = BC16channel.get_Jones_Wenzl_projection(k,N=N)
pk_hat = SNpluschannel.get_Jones_Wenzl_projection_NC(k,N=N)

kraus_ops = BC16channel.get_kraus_ops(k,l,m,N=N)
kraus_ops_hat = SNpluschannel.get_kraus_ops(k,l,m,N=N)

# Check whether the Kraus operators are a valid representation. Note that the input space is the subspace p_k H^{\otimes k}, not H^{\otimes k} itself!
sum_ops = sum( x.getH() @ x for x in kraus_ops )
sum_ops_hat = sum( x.getH() @ x for x in kraus_ops_hat )
print( np.allclose( sum_ops.todense(), pk.todense() ) , np.allclose( sum_ops_hat.todense(), pk_hat.todense() ) )

[debug] : [get_Jones_Wenzl_projection] Using memoization to find p_k for (k,N) = (2, 5).
[debug] : [get_Jones_Wenzl_projection_NC] Using memoization for (k,N) = (2,5).
[debug] : [get_Jones_Wenzl_projection] Using memoization to find p_k for (k,N) = (2, 5).
[debug] : [get_Jones_Wenzl_projection] Using memoization to find p_k for (k,N) = (2, 5).
[debug] : [get_Jones_Wenzl_projection] Using memoization to find p_k for (k,N) = (2, 5).
[debug] : [get_Jones_Wenzl_projection_NC] Using memoization for (k,N) = (2,5).
[debug] : [get_Jones_Wenzl_projection_NC] Using memoization for (k,N) = (2,5).
[debug] : [get_Jones_Wenzl_projection_NC] Using memoization for (k,N) = (2,5).
True True


As seen in the previous code section, the Kraus operators are a valid description from the input space $p_k \mathcal{H}^{\otimes k}$, not $\mathcal{H}^{\otimes k}$ itself: they do not sum to the identity, but rather to $p_k$. 

To take this into account, we can use an isometric isomorphism $V_k : \mathbb{C}^{d} \to p_k \mathcal{H}^{\otimes k}$, where $d = \dim ( p_k \mathcal{H}^{\otimes k}) = [k+1]_q$, with the properties: $V_k^* V_k = \iota_{\mathbb{C}^d}$ , and $V_k V_k^* = p_k$.

In [4]:
from qittoolbox.linalg import basis_transformation
from qittoolbox.qfunctions import qfunctions
from math import sqrt

rank_pk = qfunctions.q0_bracket(k+1,N=N)
rank_pk_hat = qfunctions.q0_bracket(2*k+1,N=sqrt(N))

# Note that the Vk and Vk_hat are *dense* np.array's, not sparse, as there is no reason to assume any sparsity!
Vk = basis_transformation.get_isometry_from_range_space_proj(pk,rank_pk)
Vk_hat = basis_transformation.get_isometry_from_range_space_proj(pk_hat,rank_pk_hat)

Vk_H = np.conj(Vk.T)
Vk_hat_H = np.conj(Vk_hat.T)

# Check whether the isometries are indeed isometries that satisfy the required conditions.
print( np.allclose( Vk_H @ Vk, np.eye(rank_pk) ) , np.allclose(Vk @ Vk_H, pk.todense()) )
print( np.allclose( Vk_hat_H @ Vk_hat, np.eye(rank_pk_hat) ) , np.allclose( Vk_hat @ Vk_hat_H, pk_hat.todense()) )


small_kraus_ops = [ Vk_H @ x @ Vk for x in kraus_ops ]
small_kraus_ops_hat = [ Vk_hat_H @ x @ Vk_hat for x in kraus_ops_hat ]

# Check whether the Kraus operators are now a valid representation on the smaller space
small_sum_ops = sum( np.conj(x.T) @ x for x in small_kraus_ops )
small_sum_ops_hat = sum( np.conj(x.T) @ x for x in small_kraus_ops_hat )
print( np.allclose( small_sum_ops, np.eye(rank_pk) ) , np.allclose( small_sum_ops_hat, np.eye(rank_pk_hat) ) )

True True
True True
True True
